In [2]:
import pandas as pd

from os import path
import pathlib ##Getting to work in Notebooks
import glob
import numpy as np
import json
import matplotlib.pyplot as plt
import math
import time

In [3]:
## Access folder where csv data is stored

## Code from previous project; altered to work for jupyter notebooks
## For code outside of notebooks, use: 
##     basepath = path.dirname(__file__)
##     filepath = path.abspath(path.join(basepath, "data"))

basepath = pathlib.Path().resolve()
filepath = path.abspath(path.join(basepath,"data"))


## https://stackoverflow.com/questions/20906474
## /import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
## Pulling from csvs when they're in a different location

files = glob.glob(filepath + "/*.csv")

sets = []

for filename in files:
    f = pd.read_csv(filename, index_col = None, header = 0)
    sets.append(f)
    
df = pd.concat(sets, axis = 0, ignore_index = True)

In [4]:
df.head(2)

,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,23,Ginger KICK! is back for the holidays with ama...,"{""id"":313,""name"":""Small Batch"",""slug"":""food/sm...",825,US,1509883503,"{""id"":990359968,""name"":""Danielle Ackley-McPhai...",USD,$,True,...,ginger-kick-holiday-cheer,https://www.kickstarter.com/discover/categorie...,True,False,successful,1510518809,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",825.0,domestic
1,322,Let's go to the movies on the L.A. River! Help...,"{""id"":298,""name"":""Movie Theaters"",""slug"":""film...",10545,US,1361914696,"{""id"":860373786,""name"":""L.A. River Revitalizat...",USD,$,True,...,la-river-bike-in-movie-theater,https://www.kickstarter.com/discover/categorie...,True,True,successful,1365700816,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",10545.0,domestic


In [4]:
# df.describe()

## Exploration

There are 210,088 rows of backer data pulled from https://webrobots.io/kickstarter-datasets/
All quantitative fields look to contain all information in some form since their respective counts are 210088. 



Looking at the backers_count field, it's interesting that the 75th percentile is at 89 while the max is at 105857. Additionally, the mean is at 145. With the mean being higher than the median, we can tell that backers_count is right skewed. This makes sense with Kickstarters and how their projects work. Incredibly popular projects will get way more than their goal and have a higher reach. 


The fields created_at, deadline, id, launched_at, state_changed_at look to be different data types than the ones presented by .describe()

In [5]:
# df.info()

There look to be several fields that are missing information.   
'blurb', 'friends', 'is_backing', 'is_starred', 'location', 'permissions', 'usd_type'

In [6]:
df[df['blurb'].isnull()]
# It looks like for these 8, blurb is alright to be left out. Some projects may just leave out
# a tiny portion of their information.

In [7]:
testset = df[df['friends'].notna()]

In [8]:
friends_value = testset[testset['friends'] != '[]'].loc[:,'friends'].item()

In [9]:
json.loads(friends_value)
## From the friends column, it looks like only one row has this information properly filled in. Others just have '[]'
## It also looks like this isn't relevant to predicting success of kickstarters. We can omit. 

In [10]:
df[df['is_backing'].notna()]['is_backing'].describe()
# The is_backing column also looks to see if the user is backing the project or not. 
# This may be related to why several values are at 252 since it's respective to the account 
# that pulled the data? This may be omitted. 
# Since the dataset provided does not have a data dictionary, we can only make assumptions about what
# the columns are

In [11]:
df[df['is_starred'].notna()]['is_starred'].describe()
# Same issue with is_starred

In [12]:
df[df['location'].isnull()]['location']
# For these, we can leave location as nulls. There are only 219 missing values and we may be able to work without them
# As of now, there is no need to clean/backfill information

In [13]:
df[df['permissions'].notna()]['permissions'].describe()
# Same issue as is_backing. It looks like no information is stored however since all 252 values are []. Can omit.

In [14]:
df[['currency', 'usd_type']].describe()

In [15]:
df[df['currency']!='USD']['usd_type'].value_counts()
# Initial assumption that usd_type related only to currencies that had USD however it seems incorrect
# Another idea could be that this relates to currency and starting country.

### Further Explorations

In [16]:
df['country'].value_counts()
# US Projects, followed by GB and CA are the most populous group. 
# US makes up about 70% of the projects. Top 3 make up 86% of projects. 

In [17]:
df['currency'].value_counts()
# Matches country values. Several countries are grouped into Euros.

In [18]:
df['goal'].describe().apply(lambda x: format(x, 'f'))

In [19]:
fig, ax = plt.subplots()
df['goal'].hist(ax=ax, bins=100, bottom=0.1)
ax.set_yscale('log')
# A majority of projects have reasonably sized goals. A few have incredibly tiny goal (0.01)
# and another few have incredibly large goals (100 million?)

In [20]:
df[df['goal'] > 90000000].head(3)
# It's interesting that these 90 million+ projects were created prior to 2018. 
# It'll also be interesting to see the categories that these projects are related to and which category pulls
# the highest project goal average. 

In [21]:
df['state'].value_counts()
## This number is different from kickstarters own 37% success rate. 
## Kickstarter also has 400k rows of data vs 200k so that could explain a discrepancy

### Data Cleaning

#### Cleaning up the times

In [22]:
# Looking at the date fields
df.head()[['created_at', 'deadline', 'launched_at', 'state_changed_at']]

It looks like these are datetime fields and the dataset is storing them in epoch time. 

In [23]:
# We can import the time module and use either the gmtime or localtime
# to converter epoch to the format we'd like.



# epochtime = df.iloc[0]['created_at']

# struct = time.gmtime(epochtime)
# time_string = time.strftime("%Y-%m-%d, %H:%M:%S", struct)
# print(epochtime)
# print(time_string)

In [4]:
def epoch_to_date(epoch):
    struct = time.gmtime(epoch)
    date_string = time.strftime("%Y-%m-%d", struct)
    return date_string
def epoch_to_time(epoch):
    struct = time.gmtime(epoch)
    time_string = time.strftime("%H:%M:%S", struct)
    return time_string

In [5]:
df['created_at_date'] = df['created_at'].apply(epoch_to_date)
df['created_at_time'] = df['created_at'].apply(epoch_to_time)

In [6]:
date_fields = ['created_at', 'deadline', 'launched_at', 'state_changed_at']
for field in date_fields:
    df[field+'_date'] = df[field].apply(epoch_to_date)
    df[field+'_time'] = df[field].apply(epoch_to_time)
# Deciding to keep the original epoch time in case. Time subtractions will be easier to do w/ epoch time than
# w/ string formatted time

#### Cleaning up categories

In [26]:
# json.loads(df['category'][0])['slug']

In [27]:
# json.loads(df['category'][1])['name']

In [7]:
def puller(value, category):
    try:
        return json.loads(value)[category]
    except:
        return 'N/A'

In [4]:
# df['category_name'] = df['category'].apply(puller, category = 'name')
# df['category_slug'] = df['category'].apply(puller, category = 'slug')

In [5]:
# df[['category', 'category_name', 'category_slug']].head()

In [8]:
to_clean = {
    'category':['name', 'slug'],
    'location':['displayable_name', 'localized_name','country', 'state'],
    'profile':['id', 'project_id', 'state'],
    'urls':['web']
}

In [9]:
for key,value in to_clean.items():
    for v in value:
        df[key+'_'+v] = df[key].apply(puller, category = v)

In [8]:
droplist = ['friends', 'is_backing', 'is_starred', 'permissions', 
           'currency_symbol', #$ related to USD, CAD, AUS etc.
           'disable_communication', #? not necessary
           'is_starrable', #? not necessary
           'photo', #not necessary. no image analysis
           'source_url'] #only discover category url, not useful for analysis since we already have category
df.drop(droplist, axis = 1, inplace = True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210088 entries, 0 to 210087
Data columns (total 55 columns):
backers_count                210088 non-null int64
blurb                        210080 non-null object
category                     210088 non-null object
converted_pledged_amount     210088 non-null int64
country                      210088 non-null object
created_at                   210088 non-null int64
creator                      210088 non-null object
currency                     210088 non-null object
currency_symbol              210088 non-null object
currency_trailing_code       210088 non-null bool
current_currency             210088 non-null object
deadline                     210088 non-null int64
disable_communication        210088 non-null bool
friends                      252 non-null object
fx_rate                      210088 non-null float64
goal                         210088 non-null float64
id                           210088 non-null int64
is_backing     